In [1]:
import gradio as gr
from joblib import Parallel, delayed
import torch
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Sklearn
import seaborn as sns
import sklearn
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Deep learning 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [2]:
def log_transform(x):
    print(x)
    return np.log1p(x)

In [3]:
class PassthroughTransformer(BaseEstimator):
    def fit(self, X, y = None):
        self.cols = X.columns
        return self
        
    def transform(self, X, y = None):
        
        X_ = X.copy()
        all_genre = {'Action',
                     'Adventure',
                     'Animation',
                     'Comedy',
                     'Crime',
                     'Documentary',
                     'Drama',
                     'Family',
                     'Fantasy',
#                      'Foreign',
                     'History',
                     'Horror',
                     'Music',
                     'Mystery',
                     'Romance',
                     'Science Fiction',
                     'TV Movie',
                     'Thriller',
                     'War',
                     'Western'}
        for gen in tqdm(all_genre):
            X_[gen] = X_['genres'].apply(lambda x: 1 if gen in x else 0)
            
        X_.drop('genres',axis=1,inplace=True)
        return X_

    def get_feature_names(self):
        return self.cols

In [4]:
Genres = ["Action", "Comedy", "Crime",'Thriller']
language = ['en','fr','es','zh']
rf_from_joblib = joblib.load('reg_rf.pkl')
cleaner_from_joblib = joblib.load('preprocessing.pkl')

In [5]:
def translate(budget,genre,language,runtime,vote_average,vote_count,release_month):
    """
    Translate the text from source lang to target lang
    """
    
    X_test = [budget,genre,language,runtime,vote_average,vote_count,release_month]
    df_new = pd.DataFrame(X_test).T
    
    df_new.columns = ['budget',
     'genres',
     'original_language',
     'runtime',
     'vote_average',
     'vote_count',
     'release_date_month']
        # Load the model from the file
    
    df_new['budget'] = df_new['budget'].astype(float)
    df_new['vote_count'] = df_new['vote_count'].astype(float)
    df_new = cleaner_from_joblib.transform(df_new)
#     print(list(df_new))
    y_pred_rf = rf_from_joblib.predict(df_new)
    y_pred_rf
    return y_pred_rf[0]

demo = gr.Interface(
    fn=translate,
    inputs=[

        gr.components.Textbox(label="budget"),
        gr.components.Dropdown(label="genres", choices=Genres),
        gr.components.Dropdown(label="Language", choices=language),
        
        gr.components.Textbox(label="runtime"),
        gr.components.Textbox(label="vote_average"),
        gr.components.Textbox(label="vote_count"),
        gr.components.Textbox(label="release_date"),
        
    ],
    outputs=["text"],
#     examples=[["Building a translation demo with Gradio is so easy!", "eng_Latn", "spa_Latn"]],
    cache_examples=False,
    title="Popularity Prediction",
#     description="This demo is a simplified version of the original [NLLB-Translator](https://huggingface.co/spaces/Narrativaai/NLLB-Translator) space"
)

demo.launch()


IMPORTANT: You are using gradio version 3.5, however version 3.14.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x24d2732f820>, 'http://127.0.0.1:7863/', None)

       budget  vote_count
0  55555555.0         3.0


  0%|          | 0/19 [00:00<?, ?it/s]

[array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , 17.83289409,  1.38629436, 10.59472955, 30.32640208,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ])]
     budget  vote_count
0  555555.0        22.0


...

[array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   1.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         6.90875478,   3.13549422,  21.15791405, 623.22048779,
         1.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ])]
   budget  vote_count
0  1000.0        22.0


  0%|          | 0/19 [00:00<?, ?it/s]

[array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   1.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         6.90875478,   3.13549422,  -4.77422808, 623.22048779,
         1.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ])]
   budget  vote_count
0  1000.0    220000.0


  0%|          | 0/19 [00:00<?, ?it/s]

[array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   1.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         6.90875478,  12.30138737,  -4.77422808, 623.22048779,
         1.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ])]
